In [1]:
# Instal dependency
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-i7mtpmr7
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-i7mtpmr7
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 2.5 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369497 sha256=d93020fb89855e821ec2d02603ccb5b7eadfcda88c326afcc47972a4f3590cb8
  Stored in directory: /tmp/pip-ephem-wheel-cache-_mnewqhu/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [2]:
# Import module
import os
import clip
import glob
import torch
import numpy as np
from PIL import Image
from tqdm import tqdm

In [3]:
all_keyframe_paths = {
    'l32_extra':dict(),
    'l33_extra':dict(),
    'l34_extra':dict(),
    'l35_extra':dict(),
    'l36_extra':dict()
}

for data_part in sorted(all_keyframe_paths.keys()):
    temp = data_part.split('_')[0].lower()
    data_part_path = f'/kaggle/input/keyframe-extra-{temp}/Keyframes/{temp.upper()}_extra'
    video_dirs = sorted(os.listdir(data_part_path))
    video_ids = [video_dir.split('_')[-1] for video_dir in video_dirs]
    for video_id, video_dir in tqdm(zip(video_ids, video_dirs)):
        keyframe_paths = sorted(glob.glob(f'{data_part_path}/{video_dir}/*.jpg'))
        all_keyframe_paths[data_part][video_id] = keyframe_paths

30it [00:04,  6.43it/s]
31it [00:03,  8.24it/s]
30it [00:05,  5.88it/s]
27it [00:05,  5.40it/s]
27it [00:04,  5.74it/s]


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model, preprocess = clip.load("ViT-B/16", device=device)

cuda


100%|███████████████████████████████████████| 335M/335M [00:19<00:00, 18.1MiB/s]


In [5]:
bs = 4
save_dir = './CLIP_features'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

for key, video_keyframe_paths in all_keyframe_paths.items():
    video_ids = sorted(video_keyframe_paths.keys())
    
    if not os.path.exists(os.path.join(save_dir, key)):
        os.mkdir(os.path.join(save_dir, key))
    
    for video_id in tqdm(video_ids):
        video_feats = []
        video_keyframe_path = video_keyframe_paths[video_id]
        
        for i in range(0, len(video_keyframe_path), bs):
            # Support batchsize inferencing
            images = []
            image_paths = video_keyframe_path[i:i+bs]
            for image_path in image_paths:
                image = preprocess(Image.open(image_path)).unsqueeze(0)
                images.append(image)
            images = torch.cat(images).to(device)

            with torch.no_grad():
                image_feats = model.encode_image(images)
            image_feats /= image_feats.norm(dim=-1, keepdim=True)

            for b in range(image_feats.shape[0]):
                video_feats.append(image_feats[b].detach().cpu().numpy().astype(np.float32).flatten())
        np.save(f'{save_dir}/{key}/{video_id}.npy', video_feats)

100%|██████████| 27/27 [12:45<00:00, 28.37s/it]
